In [1]:
from astropy.modeling import models
print(models.__file__)

/internal/1/astropy/astropy/astropy/modeling/models.py


In [2]:
from astropy.modeling.models import *
from astropy.modeling.fitting import *
import numpy as np

## Create input arrays with different shapes

In [36]:
# Create input arrays with different shapes
x = np.arange(4)
x1 = np.array([x, x]).T
x0 = np.array([x, x])
print('x: ')
print(x )
print('\n')
print('x1: ')
print(x1)
print('\n')
print('x0: ')
print(x0)

x: 
[0 1 2 3]


x1: 
[[0 0]
 [1 1]
 [2 2]
 [3 3]]


x0: 
[[0 1 2 3]
 [0 1 2 3]]


# Initialize a model set without passing `n_models`

In [9]:
pn = models.Polynomial1D(1, n_models=2)
pn

<Polynomial1D(1, c0=[ 0., 0.], c1=[ 0., 0.], n_models=2)>

In [10]:
print(pn.model_set_axis)

0


In [24]:
# This currently works:
p1 = models.Polynomial1D(1, c0=[[1,1]], c1=[[1,2]], model_set_axis=1)
print("p1: ", len(p1))
p0 = models.Polynomial1D(1, c0=[1,1], c1=[1,2], model_set_axis=0)
print("p0: ", len(p0))
p = Polynomial1D(1, c0=[1,1], c1=[1,2])
print("p: ", len(p))

# Trying this with a Gaussian1D model
g1 = Gaussian1D(amplitude=[[10,10]], mean=[[3,3]], stddev=[[1,1]], model_set_axis=1)
print("g1: ", len(g1))
g0 = Gaussian1D(amplitude=[10,10], mean=[3,3], stddev=[1,1], model_set_axis=0)
print("g0: ", len(g0))
g = Gaussian1D(amplitude=[10,10], mean=[3,3], stddev=[1,1])
print("g: ", len(g))
print("g(10): ", g(10))

p1:  2
p0:  2
p:  1
g1:  2
g0:  2
g:  1
g(10):  [  2.28973485e-10   2.28973485e-10]


There are two issues in the above examples. 
1. If `model_set_axis` is passed in the initializer, then `n_models` does not need to be passed to create a model set. But parameters need to be in the appropriate shape.
2. Using the same parameter shape as in the above example but not passing `n_models` or `model_set_axis` creates a array-parameters. This is different than a model set as it is still 1 model and setting `model_set_axis=False` does not work.

Suggested change:

- Raise an Error if `model_set_axis` is passed but `n_model` is not passed to a model initializer, i.e. always require `n_models` to initialize a model set.

- The second use case is trickier to detect. It has to be detected in the model So in the above example, Gaussian1D will have to check in the initializer that the parameters are arrays instead of scalars and raise an error. This check will need to happen in all models which use scalar parameters unless we have a `ScalarParameterModel` which they subclass.

Any thoghts on these?

# Initializing Model Sets

## 1) Initialize a model set without setting `model_set_axis`
#### Equivalent to model_set_axis=0

In [33]:
# It's set internally to its default value of 0.
p = Polynomial1D(1, n_models=2)
print(p.model_set_axis)
p

0


<Polynomial1D(1, c0=[ 0., 0.], c1=[ 0., 0.], n_models=2)>

In [37]:
# To evaluate this the 0-th axis is used to pass inputs
print(p(x0))

coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]


In [38]:
# If the inputs don't broadcast with the parameters an error is raised
#print(px1))

## 2) Initialize a model set with `model_set_axis=0`
#### Equivalent to 1)

## 3) Initialize a model set with `model_set_axis=1`

In [39]:
# 3) Initialize a model set without setting `model_set_axis=1`.
p1 = Polynomial1D(1, n_models=2, model_set_axis=1)
print(p1.model_set_axis)
p1

1


<Polynomial1D(1, c0=[[ 0., 0.]], c1=[[ 0., 0.]], n_models=2)>

In [40]:
# To evaluate this the 1-th axis is used to pass inputs
print(p1(x1))

coeffs (array([[ 0.,  0.]]), array([[ 0.,  0.]]))
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]


## Initialize a model set with model_set_axis=False

In [41]:
# 3) Initialize a model set without setting `model_set_axis=False`.
pf = Polynomial1D(1, n_models=2, model_set_axis=False)
print(pf.model_set_axis)
pf

False


<Polynomial1D(1, c0=[ 0., 0.], c1=[ 0., 0.], n_models=2)>

In [44]:
# To evaluate this the entire input is passed to each model regardless of its shape
print(pf(x1))

coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]

 [[ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]
  [ 0.  0.]]]


In [45]:
print(pf(x0))

coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]


In [52]:
x3=np.array([x,x,x])
print('x3.shape: ', x3.shape)
res=pf(x3)
print(res)
print('result.shape: ', res.shape)

x3.shape:  (3, 4)
coeffs (array([ 0.,  0.]), array([ 0.,  0.]))
[[[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]

 [[ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]
  [ 0.  0.  0.  0.]]]
result.shape:  (2, 3, 4)


# Change `model_set_axis` when a model is evaluated

The example below raises an error.
It can support the case when a model set is created by fitting a model to an array of `y`
but evaluating the fitted model set on the same independent variable. It should be fixed.

In [55]:
# p1(x0, model_set_axis=False)

# p0(x1, model_set_axis=False) already works

Currently it's not possible to fit a model when model_set_axis is set to False. 

This makes sense because a model set requires multile `y` values.

**TODO:** Catch this case and issue an intelligible error.

In [57]:
lfit = LinearLSQFitter()
#lfit(pf, x0, pf(x0))

**TODO:** Explain in the documentation that `model_set_axis=False` is valid only when a model is evaluated.

#### This means we only need to support changing `model_set_axis` from 0 or 1 to False.